## Operations Reserach Club
## Optimization with Google OR tools

Please visit the following website for the tutorials and documentation. 

https://developers.google.com/optimization/introduction/python

We will maximize the usage of the operations room of a hospital. It has multiple op-rooms and multiple departments that are doing the operations in those op-rooms.

In [1]:
from ortools.linear_solver import pywraplp

In [2]:
def create_data_model():
    """Create the data for the example."""
    data = {}
    weights = [48, 30, 42, 36, 36, 48, 42, 42, 36, 24, 30, 30, 42, 36, 36]
    values = [10, 30, 25, 50, 35, 30, 15, 40, 30, 35, 45, 10, 20, 30, 25]
    data['weights'] = weights
    data['values'] = values
    data['items'] = list(range(len(weights)))
    data['num_items'] = len(weights)
    
    num_bins = 5
    data['bins'] = list(range(num_bins))
    data['bin_capacities'] = [100, 100, 100, 100, 100]
    
    num_deps = 2
    data['deps'] = list(range(num_deps))
    data['dep_capacities'] = [100, 300]
    
    return data

In [3]:
# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver('SCIP')

In [4]:
data=create_data_model()
data

{'weights': [48, 30, 42, 36, 36, 48, 42, 42, 36, 24, 30, 30, 42, 36, 36],
 'values': [10, 30, 25, 50, 35, 30, 15, 40, 30, 35, 45, 10, 20, 30, 25],
 'items': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14],
 'num_items': 15,
 'bins': [0, 1, 2, 3, 4],
 'bin_capacities': [100, 100, 100, 100, 100],
 'deps': [0, 1],
 'dep_capacities': [100, 300]}

In [5]:
# Variables
# x[i, j] = 1 if item i is packed in bin j.
x = {}
for i in data['items']:
    for j in data['bins']:
        for k in data['deps']:
            x[(i,j,k)] = solver.IntVar(0, 1, 'x_%i_%i_%i' % (i,j,k))

In [6]:
# Constraints
# Each item can be in at most one bin.
for i in data['items']:
    solver.Add(sum(x[i, j,k] for j in data['bins'] for k in data['deps']) <= 1)
            
        
            
# The amount packed in each bin cannot exceed its capacity.
for j in data['bins']:
    solver.Add(
                sum(x[(i, j,k)] * data['weights'][i]
                    for i in data['items'] for k in data['deps']) <= data['bin_capacities'][j])
    
# The amount packed in each department cannot exceed its capacity.
for k in data['deps']:
    solver.Add(
                sum(x[(i, j,k)] * data['weights'][i]
                    for i in data['items'] for j in data['bins']) <= data['dep_capacities'][k])

In [7]:
print('Number of constraints =', solver.NumConstraints())

Number of constraints = 22


In [8]:
# Objective
objective = solver.Objective()

for i in data['items']:
    for j in data['bins']:
        for k in data['deps']:
            objective.SetCoefficient(x[(i,j,k)], data['values'][i])
objective.SetMaximization()

In [9]:
status = solver.Solve()
if status == pywraplp.Solver.OPTIMAL:
    print('Total packed value:', objective.Value())
    total_weight = 0
    for j in data['bins']:
        bin_weight = 0
        bin_value = 0

        print('Bin ', j, '\n')
        for i in data['items']:
            for k in data['deps']:
                if x[i, j,k].solution_value() > 0:
                    print('Item', i, '- weight:', data['weights'][i], ' value:',data['values'][i])
                    bin_weight += data['weights'][i]
                    bin_value += data['values'][i]
        print('Packed bin weight:', bin_weight)
        print('Packed bin value:', bin_value)

        print()
        total_weight += bin_weight
    print('Total packed weight:', total_weight)
else:
    print('The problem does not have an optimal solution.')

Total packed value: 375.0
Bin  0 

Item 2 - weight: 42  value: 25
Item 9 - weight: 24  value: 35
Item 10 - weight: 30  value: 45
Packed bin weight: 96
Packed bin value: 105

Bin  1 

Item 8 - weight: 36  value: 30
Item 13 - weight: 36  value: 30
Packed bin weight: 72
Packed bin value: 60

Bin  2 

Item 5 - weight: 48  value: 30
Item 14 - weight: 36  value: 25
Packed bin weight: 84
Packed bin value: 55

Bin  3 

Item 1 - weight: 30  value: 30
Item 3 - weight: 36  value: 50
Packed bin weight: 66
Packed bin value: 80

Bin  4 

Item 4 - weight: 36  value: 35
Item 7 - weight: 42  value: 40
Packed bin weight: 78
Packed bin value: 75

Total packed weight: 396


In [10]:
status = solver.Solve()
if status == pywraplp.Solver.OPTIMAL:
    print('Total packed value:', objective.Value(),'\n')
    total_weight = 0
    for k in data['deps']:
        dep_weight = 0
        dep_value = 0

        print('Dep ', k, '\n')
        for i in data['items']:
            for j in data['bins']:
                if x[i, j,k].solution_value() > 0:
                    print('Item', i, '- weight:', data['weights'][i], ' value:',data['values'][i])
                    dep_weight += data['weights'][i]
                    dep_value += data['values'][i]
        print('Packed department weight:', dep_weight)
        print('Packed department value:', dep_value)

        print()
        total_weight += dep_weight
    print('Total packed weight:', total_weight)
else:
    print('The problem does not have an optimal solution.')

Total packed value: 375.0 

Dep  0 

Item 1 - weight: 30  value: 30
Item 10 - weight: 30  value: 45
Item 14 - weight: 36  value: 25
Packed department weight: 96
Packed department value: 100

Dep  1 

Item 2 - weight: 42  value: 25
Item 3 - weight: 36  value: 50
Item 4 - weight: 36  value: 35
Item 5 - weight: 48  value: 30
Item 7 - weight: 42  value: 40
Item 8 - weight: 36  value: 30
Item 9 - weight: 24  value: 35
Item 13 - weight: 36  value: 30
Packed department weight: 300
Packed department value: 275

Total packed weight: 396


In [11]:
for i,j in x.items():
    if j.solution_value()>0:
        print('item ', i[0], ' is in the department ', i[2],'and in OP-room ', i[1], 'value:',data['values'][i[0]], 'weight:',data['weights'][i[0]],)

item  1  is in the department  0 and in OP-room  3 value: 30 weight: 30
item  2  is in the department  1 and in OP-room  0 value: 25 weight: 42
item  3  is in the department  1 and in OP-room  3 value: 50 weight: 36
item  4  is in the department  1 and in OP-room  4 value: 35 weight: 36
item  5  is in the department  1 and in OP-room  2 value: 30 weight: 48
item  7  is in the department  1 and in OP-room  4 value: 40 weight: 42
item  8  is in the department  1 and in OP-room  1 value: 30 weight: 36
item  9  is in the department  1 and in OP-room  0 value: 35 weight: 24
item  10  is in the department  0 and in OP-room  0 value: 45 weight: 30
item  13  is in the department  1 and in OP-room  1 value: 30 weight: 36
item  14  is in the department  0 and in OP-room  2 value: 25 weight: 36
